In [ ]:
import os
import json

task_name = 'gpt-4.1_maxiter_100_N_v0.61.0-no-hint'
eval_output_dir = '/home/v-murongma/code/OpenHands_SWE-Bench-Optimized/evaluation/evaluation_outputs/outputs/princeton-nlp__SWE-bench_Verified-test/CodeActAgent/gpt-4.1_maxiter_100_N_v0.61.0-no-hint'


def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

multirun_results = []
for dir in os.listdir(eval_output_dir):
    if task_name in dir:
        result_file = load_json(os.path.join(eval_output_dir, dir, 'report.json'))
        multirun_results.append(result_file)

submitted_ids = set(multirun_results[0]['submitted_ids'])
for result in multirun_results:
    submitted_ids = submitted_ids.intersection(set(result['submitted_ids']))
print(f"Total unique submitted IDs: {len(submitted_ids)}")

results_statistics = {}
for submitted_id in submitted_ids:
    stats = {"resolved_freq": 0, "unresolved_freq": 0, "empty_patch_freq": 0, "error_freq": 0}
    for result in multirun_results:
        if submitted_id in result['resolved_ids']:
            stats['resolved_freq'] += 1
        if submitted_id in result['unresolved_ids']:
            stats['unresolved_freq'] += 1
        if submitted_id in result['empty_patch_ids']:
            stats['empty_patch_freq'] += 1
        if submitted_id in result['error_ids']:
            stats['error_freq'] += 1
    results_statistics[submitted_id] = stats
    
# error type statistics
error_type_statistics = {}
for submitted_id in submitted_ids:
    stats = {'stuck_in_loop_freq':0, 'max_iter_error_freq':0, 'other_error_freq':0}
    for dir in os.listdir(eval_output_dir):
        if task_name in dir:
            log_content = open(os.path.join(eval_output_dir, dir, 'infer_logs', f'instance_{submitted_id}.log')).read()
            if 'maximum iteration' in log_content:
                stats['max_iter_error_freq'] += 1
            if 'AgentStuckInLoopError' in log_content:
                stats['stuck_in_loop_freq'] += 1
    error_type_statistics[submitted_id] = stats
                
    
# normalize frequencies
num_runs = len(multirun_results)
for submitted_id, stats in results_statistics.items():
    for key in stats:
        stats[key] /= num_runs
for submitted_id, stats in error_type_statistics.items():
    for key in stats:
        stats[key] /= num_runs
        
# show statistics
print("Results Statistics:")
for submitted_id, stats in results_statistics.items():
    print(f"Submitted ID: {submitted_id},\n Stats: {stats}") 
    
print("Error Type Statistics:")
for submitted_id, stats in error_type_statistics.items():
    print(f"Submitted ID: {submitted_id},\n Error Type Stats: {stats}") 

FileNotFoundError: [Errno 2] No such file or directory: '/home/v-murongma/code/OpenHands_SWE-Bench-Optimized/evaluation/evaluation_outputs/outputs/princeton-nlp__SWE-bench_Verified-test/CodeActAgent/gpt-4.1_maxiter_100_N_v0.61.0-no-hint/report.json'